In [2]:
import torch
from torch import nn

### 解决替代消失问题

#### 残差连接通过跳过一个或多个层，为梯度在网络中的流动提供了一条可替代且更短的路径。

In [7]:
class Example(nn.Module):
    def __init__(self, layer_size, ResNet):
        super().__init__()
        self.ResNet = ResNet
        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(layer_size[0], layer_size[1]),
                nn.GELU()
            ),
            nn.Sequential(
                nn.Linear(layer_size[0], layer_size[1]),
                nn.GELU()
            ),
            nn.Sequential(
                nn.Linear(layer_size[0], layer_size[1]),
                nn.GELU()
            ),
            nn.Sequential(
                nn.Linear(layer_size[0], layer_size[1]),
                nn.GELU()
            ),
            nn.Sequential(
                nn.Linear(layer_size[0], layer_size[1]),
                nn.GELU()
            )
        ])
    def forward(self, x):
        for layer in self.layers:
            layer_output = layer(x)
            if self.ResNet and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x

In [8]:
layer_sizes = [3, 3, 3, 3, 3, 1] 
sample_input = torch.tensor([[1., 0., -1.]]) 
torch.manual_seed(123) 
model_without_shortcut = Example( 
 layer_sizes, ResNet=False 
)

In [9]:
def print_gradients(model, x):
   output = model(x) 
   target = torch.tensor([[0.]]) 
   loss = nn.MSELoss() 
   loss = loss(output, target) 
   loss.backward() 
   for name, param in model.named_parameters():
    if 'weight' in name: 
        print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")

In [10]:
print_gradients(model_without_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.0005547706969082355
layers.1.0.weight has gradient mean of 0.0003348961763549596
layers.2.0.weight has gradient mean of 0.0020374569576233625
layers.3.0.weight has gradient mean of 0.004004963207989931
layers.4.0.weight has gradient mean of 0.004936755169183016


c:\Users\29611\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [12]:
torch.manual_seed(123) 
model_with_shortcut = Example( 
 layer_sizes, ResNet=True 
) 
print_gradients(model_with_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.6297153830528259
layers.1.0.weight has gradient mean of 0.5255692005157471
layers.2.0.weight has gradient mean of 0.7472164630889893
layers.3.0.weight has gradient mean of 0.4475112557411194
layers.4.0.weight has gradient mean of 0.5911622047424316


c:\Users\29611\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
